In [1]:
import cv2
import numpy as np
import os
from ultralytics import YOLO

# Cargar YOLOv8
model = YOLO('yolov8n.pt')  # Carga el modelo YOLOv8n preentrenado

# Obtener la lista de archivos en la carpeta
folder_path = "images"
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg'))]

# Inicializar una lista para almacenar todas las imágenes con objetos detectados
combined_image = None

# Iterar sobre las imágenes y detectar objetos
for image_file in image_files:
    # Cargar imagen
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)
    if image is None:
        continue  # Saltar archivos que no se puedan cargar como imágenes
    height, width, channels = image.shape

    # Detectar objetos
    results = model(image)

    # Dibujar las cajas alrededor de los objetos detectados
    for result in results:
        boxes = result.boxes.xyxy.numpy()
        confidences = result.boxes.conf.numpy()
        class_ids = result.boxes.cls.numpy().astype(int)
        labels = [model.names[int(cls)] for cls in class_ids]

        for box, confidence, label in zip(boxes, confidences, labels):
            if confidence > 0.1: # Solo dibujar si la confianza es mayor que 0.5
                x1, y1, x2, y2 = map(int, box)
                color = (0, 255, 0)
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
                fontScale = 2  # Ajusta este valor para aumentar el tamaño del texto
                text = f"{label} {confidence:.2f}"  # Agregar la confianza al texto
                cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_PLAIN, fontScale, color, 2)

    # Redimensionar la imagen para que todas tengan la misma altura
    desired_height = 800  # Especifica la altura deseada para todas las imágenes
    ratio = desired_height / height
    image_resized = cv2.resize(image, (int(width * ratio), desired_height))

    # Concatenar la imagen actual con las imágenes anteriores
    if combined_image is None:
        combined_image = image_resized
    else:
        combined_image = np.hstack((combined_image, image_resized))

# Mostrar la imagen resultante con todas las imágenes combinadas
cv2.imshow("Combined Image", combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 480x640 1 spoon, 1 apple, 1 orange, 1 mouse, 2 remotes, 1 keyboard, 1 toothbrush, 297.0ms
Speed: 0.0ms preprocess, 297.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
